# Meteo conversion optimisation for hydro models

We will go in details of how to convert meteorological data (ECMWF/ERA5): 
 * from netcdf gridded data 

 ![ECMWF](https://climate.copernicus.eu/sites/default/files/inline-images/CERRA_wind.png)
 * onto zarr unstrutured meshes format 
 
 ![global mesh](assets/ERA5_wind.gif)

The idea is to able to prepare the data to be able to use it as an input for our hydro models (SCHISM & TELEMAC) used at JRC

## 1 - Sensitivity on the chunking size

### Important! first write an empty array
> Dimensions cannot be included in both `region` and `append_dim` at the same time. To create empty arrays to fill in with `region`, use a separate call to `to_zarr()` with `compute=False.`

the first step consists in creating an initial Zarr dummy Dataset array.

In [ ]:
import dask.array as da
import numpy as np
import hvplot.pandas
import pandas as pd
import holoviews as hv
from holoviews import opts
import pandas as pd
import xarray as xr
import shutil
import os
import time

In [ ]:
# Set up parameters for the dataset
NTIMES = 744     # one month hourly
NNODES = 100000  # 100k nodes
chunk_size_time = 10  # Choose an appropriate chunk size for the time dimension

In [ ]:
zero = da.empty((NTIMES, NNODES), chunks=(chunk_size_time, NNODES))
zero

what about if we had a dimension for the variables?

In [ ]:
NVAR = 4
cube_zero = da.empty((NTIMES,NVAR, NNODES), chunks=(chunk_size_time, 1, NNODES))
cube_zero

In [ ]:
# Write the data to a temporary Zarr store
tmp = 'tmp.zarr'
ds = xr.Dataset({
    "u10": (('time', 'node'), zero),
    "v10": (('time', 'node'), zero),
    "msl": (('time', 'node'), zero),
    "tmp": (('time', 'node'), zero),
     
})
ds.to_zarr(tmp, compute=False)

we generated a metadata file, which will help us to aggregate the data later on. 

In [ ]:
!cat tmp.zarr/.zmetadata

we'll put everything in a function to calculate the time taken to write data, depending on : 
 * the chunk size on the `time` dimention 
 * the chunk size on the `node` dimention 

In [ ]:
def test_chunk_size(time_chunk, node_chunk):
    start = time.time()
    tmp = 'tmp.zarr'
    if os.path.exists(tmp):
        shutil.rmtree(tmp)
    # create attrs and file 
    zero = da.empty((NTIMES, NNODES), chunks=(time_chunk, node_chunk))
    bytes_per_element = zero.dtype.itemsize
    chunk_size_mb = time_chunk * node_chunk * bytes_per_element  / 1024 / 1000
    if chunk_size_mb > 2000: 
        return np.nan, np.nan
    else: 
        ds = xr.Dataset({
        "u10": (('time', 'node'), zero),
        "v10": (('time', 'node'), zero),
        "msl": (('time', 'node'), zero),
        "tmp": (('time', 'node'), zero), 
        }).to_zarr(tmp, compute=False)
        # Create a Pandas date range for the time coordinates
        time_range = pd.date_range(start=pd.Timestamp(2023, 1, 1), periods=NTIMES, freq='h')
        # Loop over variables
        count = 0
        for var_name in ["u10", "v10", "msl", "tmp"]:
            # Loop over time in chunks
            for t_start in range(0, NTIMES, time_chunk):
                t_end = min(t_start + time_chunk, NTIMES)
                time_chunk = time_range[t_start:t_end]
                # Loop over nodes in chunks
                for i_node in range(0, NNODES, node_chunk):
                    end_node = min(i_node + node_chunk, NNODES)
                    data_chunk = np.ones((t_end - t_start) * (end_node - i_node))
                    data_chunk = data_chunk.reshape((t_end - t_start, end_node - i_node))
                    node_chunk = np.arange(i_node, end_node)
                    coords = {'node': node_chunk, 'time':time_chunk}
                    ds = xr.Dataset({var_name: (('time', 'node'), data_chunk)}, coords=coords)
                    region = {'time': slice(t_start, t_end), 'node': slice(i_node, end_node)}
                    ds.to_zarr(tmp, mode="a-", region=region)
                    count += 1
        
        end = time.time()
        print(f"Chunk size: times:{time_chunk} x nodes:{node_chunk} - {chunk_size_mb}kB")
        return end - start, chunk_size_mb


Iterate for a range of chunk sizes

In [ ]:
chunksizes =  [10, 50, 100, 150, 200, 250, 300, 350, 400, 500]
nodessizes =  [5000, 10000, 50000, 100000] #, 500000, 1000000]

time_perf = np.zeros((len(chunksizes), len(nodessizes)))
sizes = np.zeros((len(chunksizes), len(nodessizes)))
for i_t, chunktime in enumerate(chunksizes): 
    for i_n, chunknode in enumerate(nodessizes):
        perf, size = test_chunk_size(chunktime, chunknode)
        time_perf[i_t,i_n] = perf
        sizes[i_t,i_n] = size

In [ ]:
# Convert data for pcolormesh plots into a format compatible with hvplot
time_perf_mesh_df = pd.DataFrame(time_perf, index=chunksizes, columns=nodessizes)
sizes_mesh_df = pd.DataFrame(sizes, index=chunksizes, columns=nodessizes)

# Convert data for line plot into a Pandas DataFrame
sizes_flat = sizes.ravel()
time_perf_flat = time_perf.ravel()
idx = np.argsort(sizes_flat)
line_df = pd.DataFrame({
    'Size (MB)': sizes_flat[idx],
    'Time (sec)': time_perf_flat[idx]
})

# Create hvplot pcolormesh plots
time_perf_plot = time_perf_mesh_df.hvplot.heatmap(
    x='columns', y='index', C='value', logx=True, colorbar=True, xlabel="chunk size for nodes", ylabel = "chunk size for times", 
    title='Time taken for each chunk (in seconds)', width = 500)

sizes_plot = sizes_mesh_df.hvplot.heatmap(
    x='columns', y='index', C='value', logx=True, colorbar=True, xlabel="chunk size for nodes", ylabel = "chunk size for times", 
    title='Size of each chunk (in MB)', width = 500)

# Create hvplot line plot
line_plot = line_df.hvplot.line(
    x='Size (MB)', y='Time (sec)', 
    title='Time taken vs. Size of each chunk', width = 500)

# Combine plots into a layout
layout = (time_perf_plot + sizes_plot + line_plot).cols(3)

# Apply some options for better layout
layout.opts(
    opts.HeatMap(colorbar=True),
    opts.Layout(shared_axes=False, merge_tools=False),
)
# # Display the layout
hv.output(layout)

Conclusion: that the bigger the chunk size is, the smaller is the time to process the data. 

## 2 - test with real data

We used previously the `region` parameter to append chunks to the zarr file.

According to the [docs](https://docs.xarray.dev/en/stable/user-guide/io.html#modifying-existing-zarr-stores), 
> region (`dict` or `"auto"`, optional) – Optional mapping from dimension names to integer slices along dataset dimensions to indicate the region of existing zarr array(s) in which to write this dataset’s data. For example, `{'x': slice(0, 1000), 'y': slice(10000, 11000)}` would indicate that values should be written to the region `0:1000` along `x` and `10000:11000` along `y`.

**Conclusion**: We can only enter **monotonic** slices in the `region` dictionary argument. That means we need specify the output chunks with monotomic slices of a **constant size**.

To have coherent geographic we need to reorder the nodes along their coordinates

### 2.1 load mesh file
we will use the `xarray-selafin` package to load the mesh file

In [ ]:
! poetry add xarray-selafin

In [ ]:
# load mesh file
mesh = xr.open_dataset("global-v0.slf", engine = 'selafin')
mesh

### 2.2 split and reorder along regions 

let's visualise the mesh nodes number along the lat/lon dimensions. 

In [ ]:
def plot_mesh(x, y): 
    df = pd.DataFrame({'x': x, 'y': y, 'id': np.arange(len(x))})
    im = df.hvplot.points(x='x', y='y', c='id',s=10)
    return im

x, y = mesh.x.values, mesh.y.values
plot_mesh(x,y).opts(width = 1200, height = 600,cmap='tab20c')

out of the mesher, mesh nodes are not sorted, we need to reorder them.

The methodology will proceed as following : 

1. Normalize the longitude (x) and latitude (y) coordinates of the nodes.
2. Compute the ordering weights for the nodes.
3. For each region, determine which nodes fall within that region.
4. Reorder the nodes within each region based on the computed weights.
5. Remap the connectivity of the triangles to reflect the new node ordering.

first we'll load the regions from a `json` file we previously created on `QGIS`

In [ ]:
import json
with open("world_regions.json") as f:
    json_regions = json.load(f)
json_regions

In [ ]:
import geopandas as gpd
import hvplot.pandas  # This import is necessary to use the hvplot accessor
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world_regions = gpd.read_file("world_regions.json")
countries_plot = countries.hvplot(color='k')
world_regions_plot = world_regions.hvplot(
    alpha=0.6, 
    c='id', 
    cmap='tab20c',
    hover_cols=['id','name'],)
overlay = countries_plot * world_regions_plot
overlay.opts(
    width = 1200,
    height = 600,
    )

3 functions below: 
1. `reorder_nodes_within_region`: reorder the nodes within a given region based on the computed weights.
2. `remap_connectivity`: remap the connectivity of the triangles to reflect the new node ordering.
3. `reorder_mesh`: main functions that translates input mesh to "ordered" mesh. 

In [ ]:
from typing import Tuple, List, Iterable
import numpy_indexed as npi

def remap_connectivity(
        tri: np.ndarray, 
        mapping: np.ndarray
    ) -> np.ndarray:
    """Remap the connectivity of a triangular mesh based on the new node order.

    Args:
        tri: The original connectivity array of the triangular mesh.
        mapping: The array that maps old node indices to new ones.

    Returns:
        The remapped connectivity array for the triangular mesh.
    """    
    remapped_nodes = np.arange(len(mapping))
    remapped_triface_nodes = np.c_[
        npi.remap(tri[:, 0], mapping, remapped_nodes),
        npi.remap(tri[:, 1], mapping, remapped_nodes),
        npi.remap(tri[:, 2], mapping, remapped_nodes),
    ]
    return remapped_triface_nodes


def reorder_nodes_within_region(
        x: np.ndarray, 
        y: np.ndarray, 
        region_polygon: gpd.GeoDataFrame, 
        order_wgts: np.ndarray
    ) -> np.ndarray:
    """Reorder nodes within a given region based on their weights.

    Args:
        x: The x-coordinates of the nodes.
        y: The y-coordinates of the nodes.
        region_polygon: The polygon representing the region.
        order_wgts: The weights for ordering the nodes.

    Returns:
        The indices of the reordered nodes within the given region.
    """    
    bbox = region_polygon.bounds
    points_in_region = (y >= bbox[1]) & (y <= bbox[3]) & (x >= bbox[0]) & (x <= bbox[2])
    indices_in_region = np.where(points_in_region)[0]
    order_wgts_in_region = order_wgts[indices_in_region]
    idx_sort = np.argsort(order_wgts_in_region)
    mapping = np.arange(len(x))
    mapping[indices_in_region] = indices_in_region[idx_sort]
    return indices_in_region[idx_sort]


def reorder_mesh(
        x: np.ndarray, 
        y: np.ndarray, 
        tri:np.ndarray, 
        regions: gpd.GeoDataFrame
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, List[int]]:
    """Reorder the mesh nodes and remap the connectivity for each region.

    Args:
        mesh: The dataset representing the mesh.
        regions: A GeoDataFrame representing the regions.

    Returns:
        A tuple containing the reordered x-coordinates, y-coordinates, 
        remapped connectivity, and the global sorting indices.
    """    # 1 normalise
    normalized_lon = mesh.x - np.min(mesh.x)
    normalized_lat = mesh.y - np.min(mesh.y)
    # 2 compute ordering
    order_wgts = (normalized_lon) + (180-normalized_lat) * 360
    # 3 test point in regions and fill in mapping / sorted indices
    global_sorted = []
    for ir, region in regions.iterrows():
        region_polygon = region['geometry']
        # 4. Reorder the nodes within each region 
        sorted_indices = reorder_nodes_within_region(x,y, region_polygon, order_wgts)
        global_sorted.extend(sorted_indices)
    # 5. Remap the connectivity 
    tri_out = remap_connectivity(tri, np.array(global_sorted))
    return x[global_sorted], y[global_sorted], tri_out, global_sorted


In [ ]:
x, y, tri = mesh.x.values, mesh.y.values, mesh.attrs['ikle2'] - 1
x_, y_, tri_, map_  = reorder_mesh(x, y, tri, world_regions)

test if the renumbering worked as expected: 

In [ ]:
plot_mesh(x_, y_).opts(width = 1200, height = 600, cmap = 'tab20c')

test if the triangle remapping worked as expected: 

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize = (12,6))
def is_overlapping(tris, meshx):
    PIR = 180
    x1, x2, x3 = meshx[tris].T
    return np.logical_or(abs(x2 - x1) > PIR, abs(x3 - x1) > PIR, abs(x3 - x2) > PIR)
m = is_overlapping(tri_ ,x_)
ax.tricontourf(x_, y_, tri_[~m], np.arange(len(x)),cmap = 'tab20c')

#### finalise and save the new mesh dataset: 

using [thalassa](https://github.com/ec-jrc/Thalassa/blob/b9d977cd6999e73f5ad884e9e7b96d4041b60827/thalassa/normalization.py#L27)'s `GENERIC` Format

In [ ]:
mesh_out = xr.Dataset({
    'lon': (['node'], x_),
    'lat': (['node'], y_),
    'triface_nodes': (['triface', 'three'], tri_),
    'depth': (['node'], mesh.B.isel(time=0).values[map_]),
})
mesh_out

check the depth assignation

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize = (12,6))
ax.tricontourf(x_, y_, tri_[~m], mesh_out.depth)

### 2.3 interpolate meteo and append on to zarr file

we will load ERA5 reanalysis data and test the chunking size on the `time` and `node` dimensions: 
 * for the `time` dimension, the chunking is quite  straightforward.
 * for the `node` dimension, the chunking gets more complex because we deal with 2 different meshes. we will explain why in details below

In [ ]:
# load  ERA5 reanalysis data
era5 = xr.open_dataset("era5_2023_uvp.nc")
era5

first we will the interpolation functions used in TELEMAC ( [`optim_gen-atm`](https://gitlab.pam-retd.fr/otm/telemac-mascaret/-/blob/optim-gen-atm/scripts/python3/utils/geometry.py) branch)

In [ ]:
from scipy.spatial import Delaunay, cKDTree
def get_weights(in_xy, out_xy, d=2):
    t = Delaunay(in_xy) # triangulate output mesh
    s = t.find_simplex(out_xy) 
    vert = np.take(t.simplices, np.maximum(s, 0), axis=0)  # Use max to avoid negative indices
    t_ = np.take(t.transform, np.maximum(s, 0), axis=0)
    delta = out_xy - t_[:, d]
    bary = np.einsum('njk,nk->nj', t_[:, :d, :], delta)
    wgts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
    # Points outside the out_xy
    out_idx_out = s < 0  
    if np.any(out_idx_out):
        # For points outside, find nearest neighbors
        tree = cKDTree(in_xy)
        _, in_idx_out = tree.query(out_xy[out_idx_out])
    else : 
        in_idx_out = None
    return vert, wgts, out_idx_out, in_idx_out


def interp(values, vtx, wts, out_idx_out, in_idx_out):
    res = np.einsum('nj,nj->n', np.take(values, vtx), wts)
    if in_idx_out is not None:
        res[out_idx_out] = values[in_idx_out]
    return res

here we will mimic the first function from above in order to append to a zarr store

In [ ]:

# Function to subset ERA data based on the mesh extent
def subset_era_from_mesh(
    era: xr.Dataset, 
    mesh: xr.Dataset, 
    input360: bool,
    gtype:str,
) -> xr.Dataset:
    """
    Selects a subset of ERA data that overlaps with the provided mesh's geographical extent.

    :param era: The ERA dataset from which to select a subset.
    :param mesh: The mesh dataset defining the geographical extent for the subset selection.
    :param input360: A flag indicating whether the longitude should be adjusted to a 0-360 range.
    :return: A subset of the ERA dataset that falls within the mesh's geographical extent.
    """
    xmin, xmax, ymin, ymax = mesh.lon.min(), mesh.lon.max(), mesh.lat.min(), mesh.lat.max()
    if input360:
        xmin, xmax = np.mod(xmin+360, 360), np.mod(xmax+360, 360)
        if xmax < xmin:
            xmin, xmax = 0, 360
    if gtype == "grid":
        era_chunk = era.sel(longitude=slice(xmin, xmax), latitude=slice(ymax, ymin))
    else: # for 01280 grid
        mask_lon = (era.longitude >= xmin) & (era.longitude <= xmax)
        mask_lat = (era.latitude >= ymin) & (era.latitude <= ymax)
        mask = mask_lon & mask_lat
        indices = np.where(mask)[0]
        era_chunk = era.isel(values=indices)
    return era_chunk

# Function to write meteorological data onto a mesh
def write_meteo_on_mesh(
    era_ds: xr.Dataset, 
    mesh: xr.Dataset, 
    file_out: str, 
    n_time_chunk: int, 
    n_node_chunk: int, 
    input360: bool = True,
    gtype: str = "grid",
    ttype: str = "time"
) -> None:
    """
    Writes meteorological data from an ERA dataset onto a mesh and saves the result as a zarr file.

    :param era_ds: The ERA dataset with the meteorological data.
    :param mesh: The mesh dataset representing the spatial domain.
    :param file_out: The path to the output zarr file.
    :param n_time_chunk: The size of the time chunks for processing.
    :param n_node_chunk: The size of the node chunks for processing.
    :param input360: A flag indicating whether the longitude should be adjusted to a 0-360 range.
    """
    # Create the temporary dummy zarr file
    if os.path.exists(file_out):
        shutil.rmtree(file_out)
    x, y, tri = mesh.lon.values, mesh.lat.values, mesh.triface_nodes.values
    nnodes = len(x)
    ntimes = len(era_ds.time)
    zero = da.zeros((ntimes, nnodes), chunks=(n_time_chunk, n_node_chunk))
    
    # Define coordinates and data variables for the output dataset
    coords = {
        'time': era_ds.time, 
        'node': np.arange(nnodes),
        'lon': ("node", x), 
        'lat': ("node", y),
        'triface_nodes': mesh.triface_nodes,
    }
    data_vars = {}
    for varin in era_ds.data_vars:
        data_vars[varin] = (('time', 'node'), zero)
    xr.Dataset(data_vars=data_vars, coords=coords).to_zarr(file_out, compute=False)
    
    # in the case of "tstps"
    if ttype == "step":
        t0 = pd.Timestamp(era_ds.time.values)
        seconds = era_ds.step.values / 1e9
        era_ds.time = pd.to_datetime(t0 + pd.Timedelta(seconds=seconds))

    # Iterate over nodes in chunks and write data to zarr file
    for ins in range(0, nnodes, n_node_chunk):
        end_node = min(ins + n_node_chunk, nnodes)
        node_chunk = np.arange(ins, end_node)
        mesh_chunk = mesh.isel(node=slice(ins, end_node))
        era_chunk = subset_era_from_mesh(era_ds, mesh_chunk, input360=input360, gtype=gtype)
        
        # Get weights for interpolation
        if gtype == "grid":
            nx1d = len(era_chunk.longitude)
            ny1d = len(era_chunk.latitude)
            xx = np.tile(era_chunk.longitude, ny1d).reshape(ny1d, nx1d).T.ravel()
            yy = np.tile(era_chunk.latitude, nx1d)
        else: # for O1280 grids
            xx = era_chunk.longitude
            yy = era_chunk.latitude
            era_chunk = era_chunk.drop_vars(["number", "surface"]) # useless for meteo exports
        
        in_xy = np.vstack((xx, yy)).T
        if input360:
            in_xy[:, 0][in_xy[:, 0] > 180] -= 360
        out_xy = np.vstack((mesh_chunk.lon, mesh_chunk.lat)).T
        vert, wgts, u_x, g_x = get_weights(in_xy, out_xy)  # Assuming get_weights is defined elsewhere
        
        # Interpolate and write data for each variable and time chunk
        for var_name in era_chunk.data_vars:
            for it_chunk in range(0, ntimes, n_time_chunk):
                t_end = min(it_chunk + n_time_chunk, ntimes)
                time_chunk = era_chunk.time[it_chunk:t_end]
                data_chunk = da.zeros((len(time_chunk), len(node_chunk)), chunks=(n_time_chunk, n_node_chunk))
                for it, t_ in enumerate(time_chunk):
                    tmp = np.ravel(np.transpose(era_chunk.isel(time=it_chunk+it)[var_name].values))
                    data_chunk[it,:] = interp(tmp, vert, wgts, u_x, g_x)  # Assuming interp is defined elsewhere
                coords = {'time':time_chunk, 'node': node_chunk}
                ds = xr.Dataset({var_name: (('time', 'node'), data_chunk)}, coords=coords)
                region = {'time': slice(it_chunk, t_end), 'node': slice(ins, end_node)}
                ds.to_zarr(file_out, mode="a-", region=region)


the script is ready, let's do a sensitivity on the nodes & times 

let's reduce first the dataset

In [ ]:
era_test = era5.isel(time=slice(0,1000))

In [ ]:
times_ = [50,]
nodes_ = [len(mesh_out.lon)]

time_perf = np.zeros((len(times_), len(nodes_)))
sizes = np.zeros((len(times_), len(nodes_)))
for i_t, ttime in enumerate(times_): 
    for i_n, nnode in enumerate(nodes_):
        start = time.time()
        write_meteo_on_mesh(era_test, mesh_out, "era5_2023_uvp.zarr", ttime, nnode)
        end = time.time()
        time_perf[i_t,i_n] =  end - start
        # calculate size
        bytes_per_element = zero.dtype.itemsize
        sizes[i_t,i_n] = ttime * nnode * bytes_per_element  / 1024 / 1000


visualise the performances

In [ ]:
# Convert data for pcolormesh plots into a format compatible with hvplot
time_perf_mesh_df = pd.DataFrame(time_perf, index=times_, columns=nodes_)
sizes_mesh_df = pd.DataFrame(sizes, index=times_, columns=nodes_)

options = {
    "x":'columns', 
    "y":'index', 
    "C":'value', 
    "logx" : True, 
    "logy" : True, 
    "colorbar" : True, 
    "xlabel" : "chunk size for nodes", 
    "ylabel" :  "chunk size for times", 
    "width" :  500,
}

# Convert data for line plot into a Pandas DataFrame
sizes_flat = sizes.ravel()
time_perf_flat = time_perf.ravel()
idx = np.argsort(sizes_flat)
line_df = pd.DataFrame({
    'Size (MB)': sizes_flat[idx],
    'Time (sec)': time_perf_flat[idx]
})

# Create hvplot pcolormesh plots
time_perf_plot = time_perf_mesh_df.hvplot.heatmap(**options).opts(title = "Time taken for each chunk (in seconds)")
sizes_plot = sizes_mesh_df.hvplot.heatmap(**options).opts(title = "Size of each chunk (in MB)")

# Create hvplot line plot
line_plot = line_df.hvplot.line(
    x='Size (MB)', y='Time (sec)', 
    title='Time taken vs. Size of each chunk', width = 500)

# Combine plots into a layout
layout = (time_perf_plot + sizes_plot + line_plot).cols(3)

# Apply some options for better layout
layout.opts(
    opts.HeatMap(colorbar=True),
    opts.Layout(shared_axes=False, merge_tools=False),
)
# # Display the layout
hv.output(layout)

The best combination seems to be the **max number of nodes with the minimum number of times**

check if it worked: 

In [ ]:
ds = xr.open_dataset("era5_2023_uvp.zarr", engine = "zarr")
ds

In [ ]:
import thalassa
import geoviews as gv
ds = thalassa.normalize(ds)
# Convert the node data in "Tabular" format: https://holoviews.org/user_guide/Tabular_Datasets.html
df = ds[["lon", "lat", "node"]].to_dataframe().reset_index()
# Create a geoviews Points object
gv_points = gv.Points(df, kdims=["lon", "lat"], vdims=["node"]).opts(size = 1, color='k')
# Create a Dynamic map with the variable you want, e.g. "depth"
raster_dmap = thalassa.plot(ds.sel(time = "2023-08-27", method="nearest"), "u10")
# now combine the raster with the points
combined_dmap = raster_dmap * gv_points
# plot
combined_dmap.opts(tools=["hover"], width=1200, height=600)

#### finally, convert to Selafin

In [ ]:
def remove(path):
    try:
        if os.path.isfile(path):
            os.remove(path)  # Remove a file
        elif os.path.isdir(path):
            if not os.listdir(path):  # Check if the directory is empty
                os.rmdir(path)
            else:
                shutil.rmtree(path)
    except OSError as e:
        print(f"Error: {e.strerror}")
        
def write_meteo_selafin(outpath,input_atm_zarr):
    xatm = xr.open_dataset(input_atm_zarr, engine = "zarr")  
    t0 = pd.Timestamp(ds.time.values[0])
    # Define a mapping from the original variable names to the new ones
    var_map = {
        "u10": ("WINDX", "M/S"),
        "v10": ("WINDY", "M/S"),
        "msl": ("PATM", "PASCAL"),
        "tmp": ("TAIR", "DEGREES C"),
    }
    var_attrs = {}
    for var in ds.data_vars:
        if var in var_map:
            # Attributes for the variable
            var_attrs[var] = (var_map[var][0], var_map[var][1])
    # Add global attributes after concatenation
    xatm.attrs["date_start"] = [t0.year, t0.month, t0.day, t0.hour, t0.minute, t0.second]
    xatm.attrs["ikle2"] = xatm.triface_nodes.values + 1
    xatm.attrs["variables"] = {var: attrs for var, attrs in var_attrs.items()}
    xatm = xatm.rename({"lon": "x", "lat": "y"})
    xatm = xatm.drop_vars(["triface_nodes"])
    xatm.selafin.write(outpath)
    # remove(input_atm_zarr)


In [ ]:
write_meteo_selafin("era5_2023_uvp.slf", "era5_2023_uvp.zarr")

test with O1280 grid: 

In [ ]:
O1280_test = xr.open_dataset("2023-09-01.uvp.O1280.zarr", engine = "zarr")
O1280_test

In [ ]:
nnode = len(O1280_test.longitude)
write_meteo_on_mesh(O1280_test, mesh_out, "01280_v0.zarr", 50, nnode, input360=True, gtype='tri')

In [ ]:
write_meteo_selafin("01280_202309_uvp.slf", "01280_v0.zarr")

it works !! (GIF you see at the beginning of the notebook)